In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *
from vis3d_utilities import *
from metadata import *
from data_manager import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Local Macbook Pro


In [2]:
raw_data_dirs = \
{('Nissl', 'down64'): '/home/yuncong/LGN3Datlas/m919LGN/STIF/',
}

input_image_filename_to_imagename_re_pattern_mapping = \
{('Nissl', 'down64'): '/home/yuncong/LGN3Datlas/m919LGN/STIF/(.*).tif',
}

In [3]:
image_names_all_data_dirs_flattened = set([])
image_names_all_data_dirs = {}
for vr, data_dir in raw_data_dirs.iteritems():
    if data_dir is None: continue
    image_names = set([])
    if vr in input_image_filename_to_imagename_re_pattern_mapping:
        for fn in os.listdir(data_dir):
            g = re.search(input_image_filename_to_imagename_re_pattern_mapping[vr], os.path.join(data_dir, fn))
            img_name = g.groups()[0]
            image_names.add(img_name)
            image_names_all_data_dirs_flattened.add(img_name)
    image_names_all_data_dirs[vr] = image_names
    
    
# Make sure the every image has all three channels.
for vr, img_names in image_names_all_data_dirs.iteritems():
    print vr, 'missing:' 
    print image_names_all_data_dirs_flattened - img_names
    print 

('Nissl', 'down64') missing:
set([])



In [4]:
print len(image_names_all_data_dirs_flattened), 'images are found.'

54 images are found.


In [5]:
# valid_filenames = sorted(list(image_names_all_data_dirs_flattened), key=lambda n: int(n.split('_')[-1]))

In [6]:
thumbnail_downscale_factor = 64

In [7]:
sections_to_filenames = {}
for img_name in image_names_all_data_dirs_flattened:
    sections_to_filenames[int(img_name.split('_')[-1])] = img_name

In [8]:
# Note that this could be the human-corrected version, in which case the transforms may not exist.
valid_filenames = sorted([fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)], 
                         key=lambda n: int(n.split('_')[-1]))
print len(valid_filenames), 'valid filenames'

54 valid filenames


In [9]:
tb_fmt = 'tif'

In [10]:
version = 'Nissl'

In [11]:
stack = 'M919'

In [32]:
for img_name in valid_filenames:
    in_fp = os.path.join(raw_data_dirs[('Nissl', 'down64')], img_name + '.tif')
    out_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, version='Nissl', resol='down64', fn=img_name)
    
    create_parent_dir_if_not_exists(out_fp)
    execute_command('cp %s %s' % (in_fp, out_fp))

cp /home/yuncong/LGN3Datlas/m919LGN/STIF/M919-N78--_2_0233.tif /home/yuncong/CSHL_data_processed/M919/M919_down64_Nissl/M919-N78--_2_0233_down64_Nissl.tif
0
cp /home/yuncong/LGN3Datlas/m919LGN/STIF/M919-N79--_2_0236.tif /home/yuncong/CSHL_data_processed/M919/M919_down64_Nissl/M919-N79--_2_0236_down64_Nissl.tif
0
cp /home/yuncong/LGN3Datlas/m919LGN/STIF/M919-N80--_2_0239.tif /home/yuncong/CSHL_data_processed/M919/M919_down64_Nissl/M919-N80--_2_0239_down64_Nissl.tif
0
cp /home/yuncong/LGN3Datlas/m919LGN/STIF/M919-N81--_2_0242.tif /home/yuncong/CSHL_data_processed/M919/M919_down64_Nissl/M919-N81--_2_0242_down64_Nissl.tif
0
cp /home/yuncong/LGN3Datlas/m919LGN/STIF/M919-N83--_2_0248.tif /home/yuncong/CSHL_data_processed/M919/M919_down64_Nissl/M919-N83--_2_0248_down64_Nissl.tif
0
cp /home/yuncong/LGN3Datlas/m919LGN/STIF/M919-N84--_2_0251.tif /home/yuncong/CSHL_data_processed/M919/M919_down64_Nissl/M919-N84--_2_0251_down64_Nissl.tif
0
cp /home/yuncong/LGN3Datlas/m919LGN/STIF/M919-N85--_2_0254

In [12]:
script = os.path.join(REPO_DIR, 'preprocess', 'align_consecutive_v2.py')
input_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=None, version=version, resol='down64')
output_dir = create_if_not_exists(os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_elastix_output'))

In [13]:
resol='down64'

In [14]:
t = time.time()
print 'Align...'

run_distributed("%(script)s %(stack)s \"%(input_dir)s\" \"%(output_dir)s\" \'%%(kwargs_str)s\' %(fmt)s -p %(param_fp)s -r" % \
                {'script': script,
                'stack': stack,
                'input_dir': input_dir,
                'output_dir': output_dir,
                'fmt': tb_fmt,
                 'param_fp': '/home/yuncong/Brain/preprocess/parameters/Parameters_Rigid_MutualInfo_noNumberOfSpatialSamples.txt'
                },
                kwargs_list=[{'prev_fn': valid_filenames[i-1] + '_' + resol + '_' + version, 
                              'curr_fn': valid_filenames[i] + '_' + resol + '_' + version,
                             'prev_sn': valid_filenames[i-1] ,
                             'curr_sn': valid_filenames[i] } 
                             for i in range(1, len(valid_filenames))],
                argument_type='list',
                jobs_per_node=1,
               local_only=True)

# wait_qsub_complete()

print 'done in', time.time() - t, 'seconds' # 2252 seconds full stack

Align...
rm -f ~/stderr_*; rm -f ~/stdout_*
0


Run locally.


KeyboardInterrupt: 

In [ ]:
import re

final_metric = {}
for i in range(1, len(valid_filenames)):
#     if stack == 'ChatCryoJane201710':
#         prev_fn = valid_filenames[i-1] + '-Ntb'
#         curr_fn = valid_filenames[i] + '-Ntb'
#     else:
    prev_fn = valid_filenames[i-1]
    curr_fn = valid_filenames[i]

    with open(os.path.join(output_dir, curr_fn + '_to_' + prev_fn, 'elastix.log'), 'r') as f:
        t = f.read()
        g = re.search("Final metric value  = (.*?)\n", t)
#         final_metric[(curr_fn, prev_fn)] = -float(g.groups()[0])
        if g is None:
            print i
        final_metric[i] = float(g.groups()[0])

In [ ]:
%matplotlib inline
metric_arr = np.array(final_metric.values())
plt.figure(figsize=(20,2));
plt.ylim([-1, metric_arr.max()+.1]);
plt.stem(metric_arr);
plt.title('Mutual information of pairwise registration');
plt.yticks(np.arange(-1, metric_arr.max()+.1, 0.1));
plt.xlabel('Image index');
plt.ylabel('Mutual info');